In [ ]:
import pandas as pd
data = pd.read_csv(r'C:\Users\Darkos\Desktop\Labs\WEEK_6_Mid_project_ML\datasets\meteorologicos\meteo19.csv', delimiter=';')

In [ ]:
meteo19 = pd.read_csv('meteo19.csv', delimiter=';')
meteo20 = pd.read_csv('meteo20.csv', delimiter=';')
meteo21 = pd.read_csv('meteo21.csv', delimiter=';')
meteo22 = pd.read_csv('meteo22.csv', delimiter=';')
meteo23 = pd.read_csv('meteo23.csv', delimiter=';')

agentes19 = pd.read_csv('datos201912.csv', delimiter=';')
agentes20 = pd.read_csv('datos202012.csv', delimiter=';')
agentes21 = pd.read_csv('datos202112.csv', delimiter=';')
agentes22 = pd.read_csv('datos202212.csv', delimiter=';')
agentes23 = pd.read_csv('datos202312.csv', delimiter=';')

In [ ]:
'''FUNCION METEO'''
## CONVERSION NOMBRE COLUMNAS
data.columns = [column.lower().replace(' ','_') for column in data.columns]

## MAPEO DE LAS MAGNITUDES 
mag = {80: 'rad_ultravioleta',81:'vel_Viento',82:'dir_viento', 83: 'temperatura',
    86:'humedad_rel', 87:'presion_barometrica', 88: 'rad_solar',89:'precipitacion'}

data['magnitud'] = data['magnitud'].map(mag)

## CONVERSION DE COLUMNAS DIAS A FECHA
dias_maximos = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}

dias = [f"d{str(i).zfill(2)}" for i in range(1, 32)]

## APLICACION DE MELT() 
data_melted = data.melt(
    id_vars=["provincia", "municipio", "estacion", "magnitud", "punto_muestreo", "ano", "mes"],  ## columnas que se quedan fijas
    value_vars=dias,  ## columnas que van a ser filas
    var_name="dia",  ## nombre de la nueva columna
    value_name="value"  #nombre de la columna que almacenara los valores de cada dia
)
## MANEJO DE LA COLUMNA FECHA
data_melted['dia'] = data_melted['dia'].str[1:].astype("int64")
data_melted = data_melted[data_melted['dia'] <= data_melted['mes'].map(dias_maximos)] ##filtramos para los meses que no tengan 31 dias
data_melted["fecha"] = data_melted["dia"].astype(str) + '-' + data_melted["mes"].astype(str).str.zfill(2) + '-' + data_melted["ano"].astype(str).str.zfill(2)
data_melted["fecha"] = pd.to_datetime(data_melted["fecha"], format='%d-%m-%Y', errors='coerce')
data_melted["fecha"] = data_melted["fecha"].dt.strftime('%d/%m/%Y')

data_melted.drop(columns=['ano','mes', 'dia'], inplace = True) ## dropeo de columnas


## PIVOTEAMOS LA TABLA
## Creamos un DF vacio con el nombre de las columnas que queremos
pivot_df = pd.DataFrame(columns=['estacion','punto_muestreo','temperatura','precipitacion','vel_Viento',
                                    'presion_barometrica','dir_viento','humedad_rel','rad_solar'])
## itero por todas las estaciones utilizando .unique()
for estacion in data_melted.estacion.unique():
    ##filtro por el id de estacion
    aux = data_melted[data_melted.estacion==estacion]
    ## se hace el pivot dejando como indice la fecha, y pasando como columna magnitud y valores el value
    aux_pivot = aux.pivot(index='fecha',columns='magnitud',values='value')
    ## creo una nueva columna estacion en aux pivot que almacene el iterador
    aux_pivot['estacion'] = estacion
    ## concateno el dataframe vacio y el nuevo
    pivot_df = pd.concat([pivot_df, aux_pivot])

df_estaciones = pivot_df[pivot_df['estacion'].isin([24, 54, 56, 59])]

list_columns_drop = ['punto_muestreo','rad_solar','rad_ultravioleta']

for column in list_columns_drop:
    if column in df_estaciones:
        df_estaciones = df_estaciones.drop(columns = [column])

df_estaciones.reset_index(inplace=True)
df_estaciones.rename(columns={'index':'fecha'}, inplace=True)

df_estaciones

In [ ]:
# aux = data_melted[data_melted.estacion==102]
# test = aux.pivot(index='fecha',columns='magnitud',values='value')
# test.columns

In [ ]:
pivot_df = pd.DataFrame(columns=['estacion','punto_muestreo','temperatura','precipitacion','vel_Viento','presion_barometrica','dir_viento','humedad_rel','rad_solar'])

In [ ]:
# list_agentes19 = agentes19.ESTACION.unique()
# list_agentes20 = agentes20.ESTACION.unique()
# list_agentes21 = agentes21.ESTACION.unique()
# list_agentes22 = agentes22.ESTACION.unique()
# list_agentes23 = agentes23.ESTACION.unique()

# list_meteo19 = meteo19.ESTACION.unique()
# list_meteo20 = meteo20.ESTACION.unique()
# list_meteo21 = meteo21.ESTACION.unique()
# list_meteo22 = meteo22.ESTACION.unique()
# list_meteo23 = meteo23.ESTACION.unique()

# print(list_agentes19)
# print(list_agentes20)
# print(list_agentes21)
# print(list_agentes22)
# print(list_agentes23)
# print()
# print(list_meteo19)
# print(list_meteo20)
# print(list_meteo21)
# print(list_meteo22)
# print(list_meteo23)

In [ ]:
'''FUNCION AGENTES'''
## CONVERSION NOMBRE COLUMNAS
agentes19.columns = [column.lower().replace(' ','_') for column in agentes19.columns]

## MAPEO DE LAS MAGNITUDES 
mag_agentes = {1: 'dioxido_azufre',6:'monoxido_carbono',7:'monoxido_nitrogeno', 8: 'dioxido_nitrogeno',
    9:'pm_25', 10:'pm_10', 12: 'oxido_nitrogeno',14:'ozono',20:'tolueno',30:'benceno', 35:'etilbenceno',
    37:'metaxileno',38:'paraxileno',39:'ortoxileno',42:'hidrocarburos_totales',43:'metano',
    44:'hidrocarburos_no_metanicos', 431:'metaparaxileno'}

agentes19['magnitud'] = agentes19['magnitud'].map(mag_agentes)

## CONVERSION DE COLUMNAS DIAS A FECHA
dias_maximos = {
    1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
}

dias = [f"d{str(i).zfill(2)}" for i in range(1, 32)]

## APLICACION DE MELT() 
data_melted = agentes19.melt(
    id_vars=["provincia", "municipio", "estacion", "magnitud", "punto_muestreo", "ano", "mes"],  ## columnas que se quedan fijas
    value_vars=dias,  ## columnas que van a ser filas
    var_name="dia",  ## nombre de la nueva columna
    value_name="value"  #nombre de la columna que almacenara los valores de cada dia
)
## MANEJO DE LA COLUMNA FECHA
data_melted['dia'] = data_melted['dia'].str[1:].astype("int64")
data_melted = data_melted[data_melted['dia'] <= data_melted['mes'].map(dias_maximos)] ##filtramos para los meses que no tengan 31 dias
data_melted["fecha"] = data_melted["dia"].astype(str) + '-' + data_melted["mes"].astype(str).str.zfill(2) + '-' + data_melted["ano"].astype(str).str.zfill(2)
data_melted["fecha"] = pd.to_datetime(data_melted["fecha"], format='%d-%m-%Y', errors='coerce')
data_melted["fecha"] = data_melted["fecha"].dt.strftime('%d/%m/%Y')

data_melted.drop(columns=['ano','mes', 'dia'], inplace = True) 

## PIVOTEAMOS LA TABLA
## Creamos un DF vacio con el nombre de las columnas que queremos
pivot_df = pd.DataFrame(columns=['estacion','punto_muestreo','dioxido_azufre', 'monoxido_carbono', 'monoxido_nitrogeno',
                                 'dioxido_nitrogeno', 'oxido_nitrogeno', 'pm_25', 'pm_10', 'ozono','tolueno', 'benceno', 
                                 'etilbenceno', 'hidrocarburos_totales', 'metano', 'hidrocarburos_no_metanicos'])

## itero por todas las estaciones utilizando .unique()
for estacion in data_melted.estacion.unique():
    ##filtro por el id de estacion
    aux = data_melted[data_melted.estacion==estacion]
    ## se hace el pivot dejando como indice la fecha, y pasando como columna magnitud y valores el value
    aux_pivot = aux.pivot(index='fecha',columns='magnitud',values='value')
    ## creo una nueva columna estacion en aux pivot que almacene el iterador
    aux_pivot['estacion'] = estacion
    ## concateno el dataframe vacio y el nuevo
    pivot_df = pd.concat([pivot_df, aux_pivot])

df_estaciones_agentes = pivot_df[pivot_df['estacion'].isin([24, 54, 56, 59])]



In [ ]:
data_melted['magnitud'].unique()

In [ ]:
# pivot_df = pd.DataFrame(columns=['estacion','punto_muestreo','temperatura','precipitacion','vel_Viento',
#                                  'presion_barometrica','dir_viento','humedad_rel','rad_solar'])
# for estacion in data_melted.estacion.unique():
#     aux = data_melted[data_melted.estacion==estacion]
#     test = aux.pivot(index='fecha',columns='magnitud',values='value')
#     test['estacion']=estacion
#     pivot_df = pd.concat([pivot_df, test])

# pivot_df.head()

In [ ]:
df_estaciones_agentes

In [ ]:
df_estaciones_agentes.isna().sum()

In [1]:
import script_funciones
import pandas as pd
import os

In [2]:
## CONCATENACION DE DATAFRAMES ITERANDO A TRAVES DE LAS CARPETAS

carpeta_meteo = r'C:\Users\Darkos\Desktop\Labs\WEEK_6_Mid_project_ML\datasets\meteorologicos'
carpeta_agentes = r'C:\Users\Darkos\Desktop\Labs\WEEK_6_Mid_project_ML\datasets\agentes_contaminantes'

dataframes_meteo = []
dataframes_agentes = []

for csv in os.listdir(carpeta_meteo):
    path = os.path.join(carpeta_meteo,csv)
    df = pd.read_csv(path, delimiter=';')
    ## llamo a la funcion, en una variable df
    df = script_funciones.wrangling_data_meteo(df)

    dataframes_meteo.append(df)

df_meteo = pd.concat(dataframes_meteo,ignore_index=True)

for csv in os.listdir(carpeta_agentes):
    path = os.path.join(carpeta_agentes,csv)
    df = pd.read_csv(path, delimiter=';')
    ## llamo a la funcion, en una variable df
    df = script_funciones.wrangling_data_agentes(df)

    dataframes_agentes.append(df)
df_agentes = pd.concat(dataframes_agentes,ignore_index=True)


In [3]:
## LIMPIEZA DF METEO

df_meteo['fecha_temp'] = pd.to_datetime(df_meteo['fecha'], format='%d/%m/%Y')
df_meteo['dia_mes'] = df_meteo['fecha_temp'].dt.strftime('%d-%m')
df_meteo['mes_año'] = df_meteo['fecha_temp'].dt.strftime('%m-%Y')

## extraigo mes y año para trabajar en EDA
df_meteo['mes_año'] = pd.to_datetime(df_meteo['mes_año'], format='%m-%Y')
df_meteo['mes'] = df_meteo['mes_año'].dt.month
df_meteo['año'] = df_meteo['mes_año'].dt.year
df_meteo.drop(columns='mes_año', inplace=True)

## saco valores medios por dia mes para rellenar los Nans
columns_to_fill = ['presion_barometrica', 'vel_Viento', 'precipitacion','dir_viento']

for column in columns_to_fill:
    df_meteo[column] = df_meteo.groupby('dia_mes')[column].transform(lambda x: x.fillna(x.mean())).fillna(method='ffill')

## dropeo y conversion de tipos
df_meteo.drop(columns=['fecha_temp','dia_mes'],inplace=True)
df_meteo['fecha'] = pd.to_datetime(df_meteo['fecha'], format='%d/%m/%Y')
df_meteo['fecha'] = df_meteo['fecha'].dt.strftime('%d/%m/%Y')

## agrego una nueva columna de estaciones mapeando
estacion_name = {24: 'Casa de campo', 54:'Ensanche de Vallecas', 56: 'Plaza Eliptica', 59: 'Parque Juan Carlos I'}
df_meteo['nombre_estacion'] = df_meteo['estacion'].map(estacion_name)

df_meteo['estacion'] = df_meteo['estacion'].astype('str')

## nueva columna de dir viento categorizada
def asignar_dir_cardinal(dir_viento):
    if 0 <= dir_viento <= 45 or 315 < dir_viento <= 360:
        return 'Norte'
    elif 45 < dir_viento <= 90:
        return 'Noreste'
    elif 90 < dir_viento <= 135:
        return 'Este'
    elif 135 < dir_viento <= 180:
        return 'Sureste'
    elif 180 < dir_viento <= 225:
        return 'Sur'
    elif 225 < dir_viento <= 270:
        return 'Suroeste'
    elif 270 < dir_viento <= 315:
        return 'Oeste'

# Aplicar la función a la columna 'dir_viento' para crear la nueva columna 'dir_cardinal'
df_meteo['dir_cardinal'] = df_meteo['dir_viento'].apply(asignar_dir_cardinal)

## nueva columna de estacion del año
def asignar_estacion(mes):
    if mes in [12, 1, 2]:
        return 'Invierno'
    elif mes in [3, 4, 5]:
        return 'Primavera'
    elif mes in [6, 7, 8]:
        return 'Verano'
    elif mes in [9, 10, 11]:
        return 'Otoño'

# nueva columna usando apply
df_meteo['estacion_año'] = df_meteo['mes'].apply(asignar_estacion)
        

In [4]:
## LIMPIEZA DF AGENTES CONTAMINANTES

df_agentes['fecha_temp'] = pd.to_datetime(df_agentes['fecha'], format='%d/%m/%Y')
df_agentes['dia_mes'] = df_agentes['fecha_temp'].dt.strftime('%d-%m')
df_agentes['mes_año'] = df_agentes['fecha_temp'].dt.strftime('%m-%Y')

## extraigo mes y año para trabajar en EDA
df_agentes['mes_año'] = pd.to_datetime(df_agentes['mes_año'], format='%m-%Y')
df_agentes['mes'] = df_agentes['mes_año'].dt.month
df_agentes['año'] = df_agentes['mes_año'].dt.year
df_agentes.drop(columns='mes_año', inplace=True)

## saco valores medios por dia mes para rellenar los Nans
valores_por_dia_mes = round(df_agentes.groupby('dia_mes')['ozono'].transform('mean'),1)

df_agentes['ozono'] = df_agentes['ozono'].fillna(valores_por_dia_mes)
df_agentes['ozono'] = df_agentes.groupby('dia_mes')['ozono'].fillna(method='ffill')

## dropeo y conversion de tipos
df_agentes.drop(columns=['pm_25','pm_10','fecha_temp', 'dia_mes'],inplace=True)

df_agentes['fecha'] = pd.to_datetime(df_agentes['fecha'], format='%d/%m/%Y')
df_agentes['fecha'] = df_agentes['fecha'].dt.strftime('%d/%m/%Y')

## agrego una nueva columna de estaciones mapeando
estacion_name = {24: 'Casa de campo', 54:'Ensanche de Vallecas', 56: 'Plaza Eliptica', 59: 'Parque Juan Carlos I'}
df_agentes['nombre_estacion'] = df_agentes['estacion'].map(estacion_name)

df_agentes['estacion'] = df_agentes['estacion'].astype('str')

In [5]:
## creo una lista con las targets
targets = ['monoxido_nitrogeno', 'dioxido_nitrogeno',
       'oxido_nitrogeno', 'ozono']
## itero sobre la lista para crear un file por cada target en el path especificado
for target in targets:
    df_target = df_meteo.merge(df_agentes[['fecha', 'estacion', target]], on=['fecha', 'estacion'], how='inner')

    df_target.to_csv(rf'C:\Users\Darkos\Desktop\Labs\WEEK_6_Mid_project_ML\datasets\{target}.csv', index=False)

In [7]:
df_agentes.head()

,fecha,estacion,monoxido_nitrogeno,dioxido_nitrogeno,oxido_nitrogeno,ozono,mes,año,nombre_estacion
0,01/01/2019,24,33.0,51.0,102.0,11.0,1,2019,Casa de campo
1,01/02/2019,24,1.0,3.0,5.0,77.0,2,2019,Casa de campo
2,01/03/2019,24,8.0,35.0,46.0,52.0,3,2019,Casa de campo
3,01/04/2019,24,4.0,26.0,32.0,53.0,4,2019,Casa de campo
4,01/05/2019,24,2.0,11.0,14.0,85.0,5,2019,Casa de campo


In [9]:
df_all_targets = df_meteo.merge(df_agentes[['fecha', 'estacion','monoxido_nitrogeno', 'dioxido_nitrogeno',
       'oxido_nitrogeno', 'ozono']], on=['fecha', 'estacion'], how='inner')
df_all_targets.head()

,fecha,estacion,temperatura,precipitacion,vel_Viento,presion_barometrica,dir_viento,humedad_rel,mes,año,nombre_estacion,dir_cardinal,estacion_año,monoxido_nitrogeno,dioxido_nitrogeno,oxido_nitrogeno,ozono
0,01/01/2019,24,4.4,0.0,0.69,957.0,22.0,67.0,1,2019,Casa de campo,Norte,Invierno,33.0,51.0,102.0,11.0
1,01/02/2019,24,6.2,7.0,4.38,922.0,225.0,87.0,2,2019,Casa de campo,Sur,Invierno,1.0,3.0,5.0,77.0
2,01/03/2019,24,11.1,0.0,1.14,951.0,53.0,53.0,3,2019,Casa de campo,Noreste,Primavera,8.0,35.0,46.0,52.0
3,01/04/2019,24,12.0,0.0,1.11,942.0,54.0,69.0,4,2019,Casa de campo,Noreste,Primavera,4.0,26.0,32.0,53.0
4,01/05/2019,24,18.5,0.0,1.33,940.0,267.0,41.0,5,2019,Casa de campo,Suroeste,Primavera,2.0,11.0,14.0,85.0


In [10]:
df_all_targets.to_csv(rf'C:\Users\Darkos\Desktop\Labs\WEEK_6_Mid_project_ML\datasets\df_all_targets.csv', index=False)